# Create a Kubernetes config file

This creates a kubernetes config file that you can use to access kubernetes from the command line. The authorities of your connection mirror the authority that you have in a notebook server.

You should treat the config file with the same care that you treat ssh passwords, the file is equivalent to your kubeflow password.

Copy the generated file to ~/.kube/config on a kubernetes control node. You will then be able to run Kubernetes commands from a control node.


## Update with your values
To get the control IP, SSH to the control node that you wish to use the CLI from and run:

`h=($(hostname -i)[1]); echo $h`

In [ ]:
# Update with your values
CONTROL_IP = '10.153.0.9' 
EMAIL = 'ntl@us.ibm.com' #replace with your email

In [ ]:
SERVER_IP = f'https://{CONTROL_IP}:6443'
with open('/var/run/secrets/kubernetes.io/serviceaccount/ca.crt', 'rb') as cf: 
    CERT = cf.read() 
 
with open('/var/run/secrets/kubernetes.io/serviceaccount/token', 'rb') as tf: 
    TOKEN = tf.read() 
 
with open('/var/run/secrets/kubernetes.io/serviceaccount/namespace', 'r') as nf: 
    NAMESPACE = nf.read() 
 

USER_NAME = NAMESPACE

import base64 
import yaml 
     
config_yaml = f''' 
apiVersion: v1 
clusters: 
- cluster: 
    certificate-authority-data: {base64.b64encode(CERT).decode('utf-8')} 
    server: {SERVER_IP} 
  name: cluster.local 
contexts: 
- context: 
    cluster: cluster.local 
    namespace: {NAMESPACE} 
    user: {USER_NAME} 
  name: {EMAIL} 
current-context: {EMAIL} 
kind: Config 
users: 
- name: {USER_NAME} 
  user: 
    token: {TOKEN.decode('utf-8')} 
 
''' 
 
with open('config', 'w') as kconfig: 
    kconfig.write(yaml.dump(yaml.safe_load(config_yaml))) 

## Use the config file
Copy the config file into ~/.kube/config on the control node
You should now be able to use kubectl commands!

## Sample commands

In [ ]:
# shows all pods in your namespace
!kubectl get pods

In [ ]:
# shows pods that are not running
!kubectl get pods | grep -v Running

In [ ]:
#shows pod's logs
kubectl logs <POD NAME>

In [ ]:
#show pod details including events
kubectl describe pod <POD NAME>